In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Challenge-D2").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product| 2015-08-31|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|Home Improvement|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R215C9BDXTDQOW|          4|            0|          0|   N|                Y|
|R1DTPUV1J57YHA|          5|            0|          0|   N|                Y|
| RFAZK5EWKJWOU|          5|            0|          0|   N|                Y|
|R2XT8X0O0WS1AL|          5|            0|          0|   N|                Y|
|R14GRNANKO2Y2J|          5|            0|          0|   N|                Y|
|R2BLF9VYL24LCQ|          5|            1|          1|   N|                Y|
|R1GI9UW5KJ671O|          5|            0|          0|   N|                Y|
|R2H5CEJN863M86|          5|            0|          1|   N|                Y|
| R5PPDHFOZ3SMU|          5|            0|          0|   N|                Y|
| RE1L9IENKJJ7Y|          1|            0|          0|   N|     

In [6]:
# Filter total_votes for >=20
new_vine_df = vine_df.filter("total_votes>=20")
new_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y|
| RQCC7XNYI014B|          1|           91|         92|   N|                Y|
|R2U8FRHRVW5D2C|          5|           23|         23|   N|                N|
|R3RJZC5J4VP7AT|          4|           86|         91|   N|     

In [7]:
# Filter for helpful votes / total_votes
new_vine_df2 = new_vine_df.filter("(helpful_votes/total_votes)>=0.5")
new_vine_df2.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y|
| RQCC7XNYI014B|          1|           91|         92|   N|                Y|
|R2U8FRHRVW5D2C|          5|           23|         23|   N|                N|
|R3RJZC5J4VP7AT|          4|           86|         91|   N|     

In [8]:
# Filter for reviews that are part of the vine program (i.e., paid reviews)
paid_vine_df = new_vine_df2.filter("vine=='Y'")
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R29V4UDSS053D8|          3|           33|         38|   Y|                N|
|R1I2D068WC37PA|          3|           32|         37|   Y|                N|
|R13W2U74F67QED|          5|           62|         76|   Y|                N|
|R2QI37XFOBKUGD|          4|           24|         33|   Y|                N|
|R19F60BB2DNCKN|          5|          170|        174|   Y|                N|
|R2PYNEWYYQDWHI|          5|           24|         27|   Y|                N|
|R1UEWHCPJL2XJB|          5|           70|         72|   Y|                N|
| RXTMMEBRYZB53|          5|           21|         24|   Y|                N|
|R1PXVYIYMYGUZL|          5|           21|         26|   Y|                N|
| RSFMJJXFZHX1C|          5|           24|         25|   Y|     

In [9]:
# Filter for reviews that are not part of the vine program (i.e., unpaid reviews)
unpaid_vine_df = new_vine_df2.filter("vine=='N'")
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y|
| RQCC7XNYI014B|          1|           91|         92|   N|                Y|
|R2U8FRHRVW5D2C|          5|           23|         23|   N|                N|
|R3RJZC5J4VP7AT|          4|           86|         91|   N|     

# Data for reviews part of the vine program

In [11]:
# TOTAL REVIEWS
total_paid = paid_vine_df.count()
total_paid

266

In [12]:
# 5-STAR REVIEWS
paid_five_star = paid_vine_df.filter("star_rating=='5'").count()
paid_five_star

125

In [13]:
# PERCENT
percent_paid = (paid_five_star / total_paid)*100
percent_paid

46.99248120300752

# Data for reviews not part of the vine program

In [14]:
# TOTAL REVIEWS
total_unpaid = unpaid_vine_df.count()
total_unpaid

38829

In [15]:
# 5-STAR REVIEWS
unpaid_five_star = unpaid_vine_df.filter("star_rating=='5'").count()
unpaid_five_star

18246

In [16]:
# PERCENT
percent_unpaid = (unpaid_five_star / total_unpaid)*100
percent_unpaid

46.99065131731438